# Objet : Analyse des relations entre champs des données IRVE

## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par la détection d'incohérences de relations
- l'analyse a posteriori permet de (re)trouver la logique de dépendance entre les colonnes qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 2% des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(liens actifs sur jupyter Notebook ou Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Séparation des pdc itinerance et hors itinerance](#3---Séparation-des-pdc-itinerance-et-hors-itinerance)
- [4 - Bilan initial intégrité](#4---Bilan-initial-intégrité)
- [5 - Séparation doublons pdc - date de maj](#5---Séparation-doublons-pdc---date-de-maj)
- [6 - Séparation doublons station - date de maj](#6---Séparation-doublons-station---date-de-maj)
- [7 - Synthèse](#7---Synthèse)
- [8 - Exemples d erreurs résiduelles](#8---Exemples-d-erreurs-résiduelles)
- [Annexe : Initialisation du modèle de données 'mermaid'](#Annexe-:-Initialisation-du-modèle-de-données-'mermaid')

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-20230303.csv*"

-----------------------
# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ fictif associé à une ligne du tableau*

In [171]:
from base64 import b64encode
from IPython.display import Image, display
with open('IRVE_modele.txt', 'r', encoding="utf-8") as f:
    modele = f.read()
display(Image(url="https://mermaid.ink/img/" + b64encode(modele.encode("ascii")).decode("ascii")))

In [172]:
def analyse_integrite(data, fields, pr=True):
    '''analyse les relations du DataFrame définies dans le dict fields et retourne un dict avec pour chaque contrôle la
    liste des index ko. Les résultats des contrôles sont également ajoutés sous forme de champs booléens à data'''
    analyse = Analysis(data)
    dic_res = analyse.check_relationship(fields)
    data['ok'] = True
    for name, lis in dic_res.items():
        data[name] = True
        data.loc[lis, name] = False
        data['ok'] = data['ok'] & data[name]
        if pr:
            print('{:<50} {:>5}'.format(name, len(data) - data[name].sum()))
    return dic_res

------
# 2 - Initialisation
## initialisation des données
- lecture du fichier issu de l'api

In [141]:
from datetime import datetime
import json
from observation import Ilist, Analysis
import pandas as pd
log = {'date_irve': '2023-03-03', 'file': 'consolidation-etalab-schema-irve-statique-v-2.2.0-20230303.csv',
      'chemin': 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/main/python/Validation/irve/'}
irve = pd.read_csv(log['chemin'] + log['file'], sep=',', low_memory=False)
log['len_irve'] = len(irve)
print('nombre de lignes : ', log['len_irve']) 

nombre de lignes :  51276


## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [142]:
fields = [
 
 # relation unicité des pdl
 { "name": "index",
   "relationship" : { "parent" : "id_pdc_itinerance", "link" : "coupled" }},   
 # relations inter entités
 { "name": "contact_operateur",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nom_enseigne",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "coordonneesXY",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "id_station_itinerance",
   "relationship" : { "parent" : "id_pdc_itinerance",     "link" : "derived" }},
 # relations intra entité - station
 { "name": "nom_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "implantation_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nbre_pdc",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "condition_acces",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "horaires",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "station_deux_roues",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 # relations intra entité - localisation
 { "name": "adresse_station",
   "relationship" : { "parent" : "coordonneesXY",         "link" : "derived" }} ]

# liste des champs liés à un controle (relations) et obligatoires (mandatory)
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
mandatory =  ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
           'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
           'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
           'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
           'date_maj', 'last_modified']

-----------------------
## 3 - Séparation des pdc itinerance et hors itinerance
- un peu moins de 1 % des points de charge sont hors itinerance

In [145]:
data = irve
data[['id_station_itinerance','id_pdc_itinerance']] = data[['id_station_itinerance','id_pdc_itinerance']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].reset_index()['index']
itinerance = data[~data['non_concerne']].reset_index()
itinerance_init = itinerance.loc[:, relations]
log['pdc_hors_itinerance'] = len(non_concerne)
log['pdc_en_itinerance'] = len(itinerance)
print('nombre de pdc hors itinerance : ', log['pdc_hors_itinerance'])
print('nombre de pdc en itinerance   : ', log['pdc_en_itinerance'])

nombre de pdc hors itinerance :  607
nombre de pdc en itinerance   :  50669


In [146]:
# arborescence des champs et nombre de valeurs différentes
il = Ilist.obj(itinerance_init)
print(il.tree())

-1: root-derived (50669)
   0 : index (50669)
      1 : contact_operateur (120)
      2 : nom_enseigne (1132)
      3 : coordonneesXY (16849)
      4 : adresse_station (16094)
      5 : id_station_itinerance (18535)
      6 : nom_station (15058)
      7 : implantation_station (5)
      8 : nbre_pdc (43)
      9 : condition_acces (2)
      10: horaires (453)
      11: station_deux_roues (7)
      12: id_pdc_itinerance (42368)
      13: date_maj (565)
      14: last_modified (444)


-----------------------
## 4 - Bilan initial intégrité
- 36 % des lignes sont erronées ( 18 116 )

In [147]:
# séparation données bonnes (itinerance_ok_1) et données résiduelles (itinerance_1)
res = analyse_integrite(itinerance_init, fields)
itinerance_ok_1 = itinerance_init.loc[itinerance_init.ok, relations].reset_index(drop=True)
itinerance_1 = itinerance_init.loc[~itinerance_init.ok, relations].reset_index(drop=True)
itinerance_init = itinerance_init.loc[:, relations]
log['init_ok'] = len(itinerance_ok_1)
log['init_ko'] = len(itinerance_1)
print("\nnombre d'enregistrements sans erreurs : ", log['init_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['init_ko'])
print("taux d'erreur : ", round(log['init_ko'] / log['pdc_en_itinerance'] * 100), ' %')

index - id_pdc_itinerance                          16123
contact_operateur - id_station_itinerance          10719
nom_enseigne - id_station_itinerance                7514
coordonneesXY - id_station_itinerance              11825
id_station_itinerance - id_pdc_itinerance            578
nom_station - id_station_itinerance                 1865
implantation_station - id_station_itinerance        1245
nbre_pdc - id_station_itinerance                    1458
condition_acces - id_station_itinerance               35
horaires - id_station_itinerance                    9869
station_deux_roues - id_station_itinerance         10968
adresse_station - coordonneesXY                     1360

nombre d'enregistrements sans erreurs :  32553
nombre d'enregistrements avec au moins une erreur :  18116
taux d'erreur :  36  %


-----------------------
## 5 - Séparation doublons pdc - date de maj
- la moitié des pdc en erreur sont liées aux doublons de pdc
- la suppression des doublons permet de diviser par 5 le nombre de lignes erronnées ( 2 166 )

In [148]:
# séparation doublons pdc (doublons_pdc) et données résiduelles (itinerance_2)
itinerance_1['doublons_pdc'] = itinerance_1.sort_values(by='date_maj').duplicated('id_pdc_itinerance', keep='last')

doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)['index']
itinerance_2 = itinerance_1[~itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_1 = itinerance_1.loc[:, relations]
log['doublons_pdc'] = len(doublons_pdc)
log['sans_doublons_pdc'] = len(itinerance_2)
print('nombre de doublons pdc : ', log['doublons_pdc'])
print('nombre de pdc sans doublon   : ', log['sans_doublons_pdc'])

nombre de doublons pdc :  8301
nombre de pdc sans doublon   :  9815


In [149]:
# séparation données bonnes (itinerance_ok_3) et données résiduelles (itinerance_3)
res = analyse_integrite(itinerance_2, fields)
itinerance_ok_3 = itinerance_2.loc[itinerance_2.ok, relations].reset_index(drop=True)
itinerance_3 = itinerance_2.loc[~itinerance_2.ok, relations].reset_index(drop=True)
itinerance_2 = itinerance_2.loc[:, relations]
log['etape3_ok'] = len(itinerance_ok_3)
log['etape3_ko'] = len(itinerance_3)
print("\nnombre d'enregistrements sans erreurs : ", log['etape3_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape3_ko'])

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance            200
nom_enseigne - id_station_itinerance                 457
coordonneesXY - id_station_itinerance                624
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  204
implantation_station - id_station_itinerance         136
nbre_pdc - id_station_itinerance                     513
condition_acces - id_station_itinerance                5
horaires - id_station_itinerance                     194
station_deux_roues - id_station_itinerance           521
adresse_station - coordonneesXY                      587

nombre d'enregistrements sans erreurs :  7649
nombre d'enregistrements avec au moins une erreur :  2166


-----------------------
## 6 - Séparation doublons station - date de maj
- 20% des erreurs résiduelles sont liées au mélange d'anciens et de nouveaux pdc
- la suppression des anciens pdc permet de réduire de 25% le nombre de lignes erronnées ( 1 616 )
- les dernières erreurs correspondent à 468 stations associées à 32 opérateurs et sont liées à des causes multiples

In [150]:
# séparation doublons stations (doublons_stat_maj) et données résiduelles (itinerance_4)
itinerance_3['stat_maj'] = itinerance_3.id_station_itinerance + itinerance_3.date_maj
stat_maj_unique = itinerance_3.sort_values(by='stat_maj').drop_duplicates('id_station_itinerance', keep='last')
itinerance_3['last_stat_maj'] = itinerance_3['stat_maj'].isin(stat_maj_unique['stat_maj'])

doublons_stat_maj = itinerance_3[~itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)['index']
itinerance_4 = itinerance_3[itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)
itinerance_3 = itinerance_3.loc[:, relations]
log['doublons_station'] = len(doublons_stat_maj)
log['sans_doublons_station'] = len(itinerance_4)
print('nombre de doublons stations : ', log['doublons_station'])
print('nombre de pdc sans doublon   : ', log['sans_doublons_station'])

nombre de doublons stations :  375
nombre de pdc sans doublon   :  1791


In [151]:
# séparation données bonnes (itinerance_ok_5) et données résiduelles (itinerance_5 / itinerance_5_full)
res = analyse_integrite(itinerance_4, fields)
itinerance_ok_5 = itinerance_4.loc[itinerance_4.ok, relations].reset_index(drop=True)
itinerance_5_full = itinerance_4.loc[~itinerance_4.ok].reset_index(drop=True)
itinerance_5 = itinerance_5_full.loc[:, relations]
#itinerance_4 = itinerance_4.loc[:, relations]
log['etape5_ok'] = len(itinerance_ok_5)
log['etape5_ko'] = len(itinerance_5)
print("\nnombre d'enregistrements sans erreurs : ", log['etape5_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape5_ko'])

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              0
nom_enseigne - id_station_itinerance                 267
coordonneesXY - id_station_itinerance                216
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                   40
implantation_station - id_station_itinerance          23
nbre_pdc - id_station_itinerance                     452
condition_acces - id_station_itinerance                0
horaires - id_station_itinerance                      53
station_deux_roues - id_station_itinerance           325
adresse_station - coordonneesXY                      414

nombre d'enregistrements sans erreurs :  175
nombre d'enregistrements avec au moins une erreur :  1616


In [152]:
# structure des données résiduelles et nombre de valeurs
il = Ilist.obj(itinerance_5)
print(il.tree())

-1: root-derived (1616)
   0 : index (1616)
      2 : nom_enseigne (317)
         9 : condition_acces (2)
      3 : coordonneesXY (413)
      4 : adresse_station (468)
         1 : contact_operateur (32)
      5 : id_station_itinerance (468)
         13: date_maj (82)
      6 : nom_station (432)
      7 : implantation_station (4)
      8 : nbre_pdc (21)
      10: horaires (49)
      11: station_deux_roues (5)
      12: id_pdc_itinerance (1616)
      14: last_modified (54)


In [180]:
# nb maxi d'erreurs
itinerance_4['somme'] = 0
for name in res.keys():
    itinerance_4['somme'] += 1 - itinerance_4[name]
erreurs = max(itinerance_4['somme'])
maxi = itinerance_4[itinerance_4.somme >= erreurs]
print("nombre d'enregistrements avec ", erreurs, " erreurs : ", len(maxi))

nombre d'enregistrements avec  4  erreurs :  12


-----------------------
## 7 - Synthèse


In [181]:
# consolidation des données
itinerance['doublons_stat_maj'] = itinerance['index'].isin(doublons_stat_maj)
itinerance['doublons_pdc'] = itinerance['index'].isin(doublons_pdc)
itinerance['lignes_a_corriger'] = itinerance['index'].isin(itinerance_5['index'])
itinerance['doublons_a_supprimer'] = itinerance['doublons_stat_maj'] | itinerance['doublons_pdc']
itinerance['lignes_ko'] = itinerance['doublons_a_supprimer'] | itinerance['lignes_a_corriger']
print('total des lignes à corriger : ', itinerance['lignes_a_corriger'].sum())
itinerance_doublons = itinerance[itinerance['doublons_a_supprimer']].reset_index(drop=True)
print('total des doublons à supprimer : ', len(itinerance_doublons))
itinerance_ok = itinerance[~itinerance['lignes_ko']].reset_index(drop=True)
print('nombre de pdc avec controles ok : ', len(itinerance_ok))

total des lignes à corriger :  1616
total des doublons à supprimer :  8676
nombre de pdc avec controles ok :  40377


In [182]:
#fichier csv des lignes résiduelles à traiter (IRVE_residuel)
#fichier csv des données itinerance avec indicateur des données à corriger ou à ignorer (IRVE_itinerance_complet)
#fichier csv des données itinerance valides (IRVE_itinerance_valide)
#fichier csv des doublons (IRVE_itinerance_doublons)
extension = log['date_irve'] +'.csv'
itinerance_5_full.to_csv('IRVE_residuel' + extension)
itinerance.to_csv('IRVE_itinerance_complet' + extension)
itinerance_ok.to_csv('IRVE_itinerance_valide' + extension)
itinerance_doublons.to_csv('IRVE_itinerance_doublons' + extension)
log['IRVE_residuel' + extension] = len(itinerance_5_full)
log['IRVE_itinerance_complet' + extension] = len(itinerance)
log['IRVE_itinerance_valide' + extension] = len(itinerance_ok)
log['IRVE_itinerance_doublons' + extension] = len(itinerance_doublons)

In [183]:
# vérification de l'intégrité
print('\nbilan intégrité :')
res = analyse_integrite(itinerance_ok.loc[:, relations], fields, pr=False)
log['bilan_erreurs'] = sum([len(controle) for controle in res.values()])
log['date'] = datetime.now().isoformat()
print('\nerreurs : ', log['bilan_erreurs'])


bilan intégrité :

erreurs :  0


In [184]:
with open('logfile.txt', 'a', encoding="utf-8") as f:
    f.write(json.dumps(log) + '\n')
print(log)

{'date_irve': '2023-03-03', 'file': 'consolidation-etalab-schema-irve-statique-v-2.2.0-20230303.csv', 'chemin': 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/main/python/Validation/irve/', 'len_irve': 51276, 'pdc_hors_itinerance': 607, 'pdc_en_itinerance': 50669, 'init_ok': 32553, 'init_ko': 18116, 'doublons_pdc': 8301, 'sans_doublons_pdc': 9815, 'etape3_ok': 7649, 'etape3_ko': 2166, 'doublons_station': 375, 'sans_doublons_station': 1791, 'etape5_ok': 175, 'etape5_ko': 1616, 'IRVE_residuel2023-03-03.csv': 1616, 'IRVE_itinerance_complet2023-03-03.csv': 50669, 'IRVE_itinerance_valide2023-03-03.csv': 40377, 'IRVE_itinerance_doublons2023-03-03.csv': 8676, 'bilan_erreurs': 0, 'date': '2023-04-14T16:44:14.053451'}


In [185]:
# structure des données bonnes
il = Ilist.obj(itinerance_ok.loc[:, mandatory])
print(il.tree())

NameError: name 'mandatory' is not defined

-----------------------
## 8 - Exemples d erreurs résiduelles 

### Erreurs multiples 
- exemple avec le nombre maximal d'erreurs (4) (2 stations soit 12 pdc avec : adresse, nom, nbre_pdc et deux-roues erronés)

In [ ]:
maxi.loc[:, relations]

### Cohérence implantation_station - id_station
- 23 pdc sont liés à une erreur de choix d'implantation (3 stations, 1 opérateur)

In [ ]:
itinerance_4.loc[~itinerance_4['implantation_station - id_station_itinerance'], relations]

### Cohérence nom_station - id_station
- 40 pdc sont associés à une station avec un nom non cohérent

In [ ]:
itinerance_4.loc[~itinerance_4['nom_station - id_station_itinerance'], relations]

### Cohérence adresse - coordonnées
- 414 pdc ont une adresse non cohérente avec les coordonnées géographiques

In [ ]:
itinerance_4.loc[~itinerance_4['adresse_station - coordonneesXY'], relations]

- exemple : une station avec plusieurs adresses -> erreur de saisie ?

In [ ]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[0.193942, 49.544211]', relations]

- exemple : une station avec plusieurs adresses -> erreur de saisie ?

In [ ]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[2.87930851314442, 48.94679007929618]', relations]

### Cohérence station - coordonnées
- 216 pdc sont associés à des stations avec plusieurs coordonnées

In [ ]:
itinerance_5.loc[~itinerance_5['coordonneesXY - id_station_itinerance'], relations]

- exemple : une station avec plusieurs coordonnées -> incompréhension distinction station / pdc

In [ ]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR073PCAMAIEUFR', relations]

- exemple : une station avec plusieurs coordonnées -> erreur de saisie

In [ ]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FRV75PPX0704', relations]

- exemple : une station par point de charge avec nbre_pdc = 1

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[0.191899, 49.506711]', relations]